<a href="https://www.kaggle.com/code/omarelhakim0/eeg-emotion-classification-on-seediv?scriptVersionId=282211457" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install torch_scatter torcheeg 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.4/251.4 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.5/231.5 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of mne to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of wfdb to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.1/295.1 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# !rm -rf tmp_out

In [3]:
import torch
import torch.nn as nn
import torch.nn.utils as utils
from torch.utils.data import DataLoader, Subset
from torcheeg.models import TSCeption
from torcheeg.datasets import SEEDIVDataset
from torcheeg import transforms
import scipy.signal as signal
import random

In [4]:
# 1. Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def BandPassFilter(eeg_data):
    b, a = signal.butter(4, Wn=[1.0, 75.0], btype='bandpass', fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [6]:
def Notch(eeg_data):
    b, a = signal.iirnotch(w0=50.0, Q=30.0, fs=200)
    return signal.filtfilt(b, a, eeg_data, axis=-1)

In [7]:
# 2. Define Preprocessing
t_transform = transforms.Compose([
    transforms.Lambda(BandPassFilter),
    transforms.Lambda(Notch),
    transforms.BaselineRemoval(),
    transforms.MeanStdNormalize(),
    transforms.To2d()
    
])

In [8]:
# 3. Load Data
dataset = SEEDIVDataset(
    io_path='./tmp_out/seed_iv',
    root_path='/kaggle/input/seed-iv/eeg_raw_data',
    offline_transform=t_transform,
    label_transform=transforms.Compose([
        transforms.Select('emotion'),
    ]),
    chunk_size=800,  # 4 seconds
    num_worker=4
)

[2025-11-27 13:57:36] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to ./tmp_out/seed_iv.
[2025-11-27 13:57:36] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]:   0%|          | 0/45 [00:00<?, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 0it [00:00, ?it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 1it [00:04,  4.59s/it]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 7it [00:04,  2.02it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 14it [00:04,  4.80it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 21it [00:04,  8.39it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 27it [00:05, 12.16it/s]
[RECORD /kaggle/input/seed-iv/eeg_raw_data/1/4_20151111.mat]: 34it [00:05, 17.53it/s]
[RECORD /kaggle/input/s

In [9]:
import pandas as pd

# 1. Get the metadata DataFrame
df = dataset.info

# 2. Count the segments for each emotion
# 0: Neutral, 1: Sad, 2: Fear, 3: Happy
counts = df['emotion'].value_counts().sort_index()
total = len(df)

print(f"Total Segments: {total}")
print("-" * 30)
print("Count per Emotion:")
print(counts)

print("-" * 30)
print("Percentage per Emotion:")
percentages = (counts / total) * 100
print(percentages.round(2))

# 3. Check for Imbalance
# If the difference between max and min is > 10%, we might need a WeightedSampler
max_pct = percentages.max()
min_pct = percentages.min()

if (max_pct - min_pct) > 10:
    print(f"\n⚠️ WARNING: Data is IMBALANCED (Diff: {max_pct - min_pct:.2f}%)")
    print("Consider using a WeightedRandomSampler.")
else:
    print(f"\n✅ Data is reasonably BALANCED (Diff: {max_pct - min_pct:.2f}%)")

Total Segments: 37575
------------------------------
Count per Emotion:
emotion
0    10170
1    10245
2     9225
3     7935
Name: count, dtype: int64
------------------------------
Percentage per Emotion:
emotion
0    27.07
1    27.27
2    24.55
3    21.12
Name: count, dtype: float64

✅ Data is reasonably BALANCED (Diff: 6.15%)


In [10]:
# Split by Trial ID
# SEED-IV has 24 trials (videos) per session.
# 80% of VIDEOS for training (19 videos), 20% for testing (5 videos).
all_trial_ids = list(range(1, 25))

random.seed(42)
test_trial_ids = random.sample(all_trial_ids, 5)
train_trial_ids = [t for t in all_trial_ids if t not in test_trial_ids]

train_indices = df[df['trial_id'].isin(train_trial_ids)].index.tolist()
test_indices = df[df['trial_id'].isin(test_trial_ids)].index.tolist()

# Create Subsets & Loaders
train_set = Subset(dataset, train_indices)
test_set = Subset(dataset, test_indices)

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)

In [11]:
model = TSCeption(
    num_electrodes=62,  # SEED-IV has 62 channels
    num_classes=4,      # Neutral, Sad, Fear, Happy
    sampling_rate=200,  # 200 Hz
    num_T=15,           # Length of temporal filters (Standard default)
    num_S=15,           # Length of spatial filters (Standard default)
    hid_channels=32,          # Hidden layer size
    dropout=0.5
).to(device)

In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
# Scheduler: If validation loss doesn't go down for 3 epochs, cut LR by half
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
criterion = nn.CrossEntropyLoss()

In [13]:
best_val_loss = float('inf')
patience_limit = 10  # Increased patience because we are learning slower
counter = 0

for epoch in range(50): # Run longer because LR is smaller
    # --- TRAINING ---
    model.train()
    train_loss = 0
    correct_train = 0
    total_train = 0
    
    for batch in train_loader:
        X = batch[0].to(device).float()
        y = batch[1].to(device).long()
        
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        
        # --- CHANGE 2: Gradient Clipping ---
        # This prevents the "Loss = 39.0" explosions
        utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += y.size(0)
        correct_train += (predicted == y).sum().item()
        
    avg_train_loss = train_loss / len(train_loader)
    train_acc = (correct_train / total_train) * 100

    # --- VALIDATION ---
    model.eval()
    val_loss = 0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for batch in test_loader:
            X = batch[0].to(device).float()
            y = batch[1].to(device).long()
            
            outputs = model(X)
            loss = criterion(outputs, y)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += y.size(0)
            correct_val += (predicted == y).sum().item()
            
    avg_val_loss = val_loss / len(test_loader)
    val_acc = (correct_val / total_val) * 100
    
    # Update Scheduler
    scheduler.step(avg_val_loss)

    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f} ({train_acc:.1f}%) | Val Loss={avg_val_loss:.4f} ({val_acc:.1f}%)")

    # --- SAVE BEST ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'best_eegnet_stable.pth')
        print(f"  --> Saved Best Model (Acc: {val_acc:.2f}%)")
        counter = 0
    else:
        counter += 1
        if counter >= patience_limit:
            print("Early Stopping.")
            break

Epoch 1: Train Loss=1.3561 (31.8%) | Val Loss=1.3732 (32.3%)
  --> Saved Best Model (Acc: 32.29%)
Epoch 2: Train Loss=1.2949 (39.1%) | Val Loss=1.3652 (35.6%)
  --> Saved Best Model (Acc: 35.56%)
Epoch 3: Train Loss=1.2464 (42.6%) | Val Loss=1.3336 (39.3%)
  --> Saved Best Model (Acc: 39.34%)
Epoch 4: Train Loss=1.2076 (44.8%) | Val Loss=1.3449 (40.2%)
Epoch 5: Train Loss=1.1745 (46.9%) | Val Loss=1.3142 (40.4%)
  --> Saved Best Model (Acc: 40.37%)
Epoch 6: Train Loss=1.1475 (48.7%) | Val Loss=1.3374 (40.6%)
Epoch 7: Train Loss=1.1226 (50.3%) | Val Loss=1.3016 (42.8%)
  --> Saved Best Model (Acc: 42.82%)
Epoch 8: Train Loss=1.0988 (51.9%) | Val Loss=1.2870 (43.9%)
  --> Saved Best Model (Acc: 43.91%)
Epoch 9: Train Loss=1.0734 (53.6%) | Val Loss=1.2676 (44.0%)
  --> Saved Best Model (Acc: 43.97%)
Epoch 10: Train Loss=1.0491 (54.9%) | Val Loss=1.2668 (45.3%)
  --> Saved Best Model (Acc: 45.27%)
Epoch 11: Train Loss=1.0273 (56.3%) | Val Loss=1.2592 (44.5%)
  --> Saved Best Model (Acc: 44